# Notebook 06: Full Inference for Submission
**Mục tiêu:**
1. Chạy pipeline dự đoán trên **TẤT CẢ** các tập dữ liệu: Train, Validation, và Test.
2. Sinh ra file `pred.json` cho từng bài báo.
3. Sắp xếp kết quả vào đúng cấu trúc thư mục để nộp bài (Submission Ready).

**Cấu trúc Output:**
```bash
submission_final/
├── <Student_ID>/
│   ├── <paper_id_1>/
│   │   └── pred.json
│   ├── <paper_id_2>/
│   │   └── pred.json
...
```

In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import joblib
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Setup path để import src module
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import từ src.ml module (Sử dụng lại các hàm đã refactor)
from src.ml import (
    load_json,
    save_json,
    normalize_text_basic,
    compute_pairwise_features,
    compute_tfidf_cosine_single,
    parse_bibtex_smart,
    transform_to_paper_based
)

pd.set_option('display.max_columns', None)
print("✅ Libraries và src.ml modules imported successfully!")


✅ Libraries và src.ml modules imported successfully!


In [2]:
# --- CẤU HÌNH QUAN TRỌNG ---

# 1. Thông tin sinh viên (Để tạo folder nộp bài)
STUDENT_ID = "23127011" 

# 2. Đường dẫn dữ liệu
DATASET_ROOT = '../../dataset_final'
PARTITIONS = ['train', 'validation', 'test']

# 3. Đường dẫn Model
MODEL_PATH = '../../dataset_final/models/best_matcher.pkl'
FEATURE_NAME_PATH = '../../dataset_final/models/feature_names.pkl'

# 4. Thư mục Output
SUBMISSION_DIR = f'submission_final/{STUDENT_ID}'

if not os.path.exists(SUBMISSION_DIR):
    os.makedirs(SUBMISSION_DIR)
    print(f"📁 Created submission directory: {os.path.abspath(SUBMISSION_DIR)}")
else:
    print(f"📁 Saving to existing directory: {os.path.abspath(SUBMISSION_DIR)}")


📁 Created submission directory: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_final\23127011


In [3]:
# --- HELPER FUNCTION: Feature Calculation Wrapper ---
def compute_features_with_tfidf(row):
    """Compute features bao gồm TF-IDF cosine."""
    feats = compute_pairwise_features(row)
    
    # Thêm TF-IDF cosine cho single pair
    q_tit = normalize_text_basic(row.get('bib_title', ''))
    c_tit = normalize_text_basic(row.get('cand_title', ''))
    feats['feat_title_tfidf_cosine'] = compute_tfidf_cosine_single(q_tit, c_tit)
    
    return feats

print("✅ Feature wrapper ready.")


✅ Feature wrapper ready.


In [4]:

# --- LOAD MODEL ---
try:
    model = joblib.load(MODEL_PATH)
    feature_names = joblib.load(FEATURE_NAME_PATH)
    print(f"✅ Model loaded: {type(model).__name__}")
    print(f"📝 Expected features: {len(feature_names)}")
except FileNotFoundError:
    print("❌ CRITICAL ERROR: Model file not found. Cannot proceed.")
    model = None
    feature_names = []


✅ Model loaded: XGBClassifier
📝 Expected features: 13



## Main Inference Loop
Vòng lặp này sẽ:
1. Duyệt qua từng Partition (Train -> Valid -> Test).
2. Load `labels.json` của partition đó.
3. Transform sang cấu trúc Paper-based.
4. Chạy Ranking Model.
5. Lưu file `pred.json` vào folder tương ứng với Paper ID.


In [5]:
# Biến để thống kê tổng kết
summary_stats = {
    'train': {'papers': 0, 'mrr': 0, 'queries': 0},
    'validation': {'papers': 0, 'mrr': 0, 'queries': 0},
    'test': {'papers': 0, 'mrr': 0, 'queries': 0}
}

print("🚀 STARTING FULL INFERENCE PIPELINE...\n")

for partition in PARTITIONS:
    print(f"🔵 Processing Partition: [{partition.upper()}]")
    
    # 1. Load Data
    label_file = os.path.join(DATASET_ROOT, partition, 'labels.json')
    if not os.path.exists(label_file):
        print(f"   ⚠️ Warning: File {label_file} not found. Skipping.")
        continue
        
    raw_data = load_json(label_file)
    if not raw_data:
        print("   ⚠️ Empty data. Skipping.")
        continue

    # 2. Transform Data
    # Sử dụng hàm từ module src.ml
    papers_db = transform_to_paper_based(raw_data, parse_bibtex_smart)
    summary_stats[partition]['papers'] = len(papers_db)
    
    print(f"   ↳ Found {len(papers_db)} papers. Ranking now...")
    
    # 3. Ranking Loop
    part_mrr_sum = 0
    part_query_count = 0
    
    for paper_id, data in tqdm(papers_db.items(), desc=f"   Ranking {partition}"):
        queries = data['queries']
        candidates_dict = data['candidates']
        candidates_list = list(candidates_dict.values())
        
        # Nếu paper không có candidate nào (edge case)
        if not candidates_list:
            continue
            
        # Init Output Structure cho pred.json
        pred_output = {
            "partition": partition,
            "groundtruth": {},
            "prediction": {}
        }
        
        # Duyệt từng Query
        for query in queries:
            bib_key = query['key']
            true_id = query['true_id']
            
            # Groundtruth
            pred_output['groundtruth'][bib_key] = true_id
            
            # Pairing
            pairs = []
            for cand in candidates_list:
                row = {}
                row.update(query)
                row.update(cand)
                pairs.append(row)
            
            # Feature Engineering
            feats_list = [compute_features_with_tfidf(p) for p in pairs]
            df_feats = pd.DataFrame(feats_list)
            
            # Prediction
            scores = []
            if model:
                # Fill missing cols
                for col in feature_names:
                    if col not in df_feats.columns: df_feats[col] = 0.0
                
                X_input = df_feats[feature_names]
                scores = model.predict_proba(X_input)[:, 1]
            else:
                scores = np.random.rand(len(pairs))
            
            # Ranking & Top 5
            ranked_candidates = []
            for idx, score in enumerate(scores):
                ranked_candidates.append((candidates_list[idx]['cand_id'], score))
            
            ranked_candidates.sort(key=lambda x: x[1], reverse=True)
            top_5 = [x[0] for x in ranked_candidates[:5]]
            
            # Save Prediction
            pred_output['prediction'][bib_key] = top_5
            
            # MRR Calculation (Internal check)
            if true_id in top_5:
                rank = top_5.index(true_id) + 1
                part_mrr_sum += 1.0 / rank
            part_query_count += 1
            
        # --- 4. SAVE pred.json ---
        # Tạo folder cho Paper ID: submission_final/<Student_ID>/<paper_id>/
        # Lưu ý: paper_id có thể chứa ký tự '/' (ví dụ: 2005.12345), cần xử lý nếu hệ điều hành không chịu
        # Tuy nhiên Arxiv ID thường an toàn hoặc là yymm-id.
        safe_paper_id = str(paper_id).strip() 
        paper_dir = os.path.join(SUBMISSION_DIR, safe_paper_id)
        
        if not os.path.exists(paper_dir):
            os.makedirs(paper_dir)
            
        save_path = os.path.join(paper_dir, 'pred.json')
        save_json(pred_output, save_path)
    
    # Update Stats
    if part_query_count > 0:
        summary_stats[partition]['mrr'] = part_mrr_sum / part_query_count
        summary_stats[partition]['queries'] = part_query_count
    
    print(f"   ✅ Finished {partition}. Saved pred.json files.")
    print("-" * 50)


🚀 STARTING FULL INFERENCE PIPELINE...

🔵 Processing Partition: [TRAIN]
   ↳ Found 292 papers. Ranking now...


   Ranking train: 100%|██████████| 292/292 [33:16<00:00,  6.84s/it]  


   ✅ Finished train. Saved pred.json files.
--------------------------------------------------
🔵 Processing Partition: [VALIDATION]
   ↳ Found 39 papers. Ranking now...


   Ranking validation: 100%|██████████| 39/39 [04:00<00:00,  6.18s/it]


   ✅ Finished validation. Saved pred.json files.
--------------------------------------------------
🔵 Processing Partition: [TEST]
   ↳ Found 39 papers. Ranking now...


   Ranking test: 100%|██████████| 39/39 [01:15<00:00,  1.94s/it]

   ✅ Finished test. Saved pred.json files.
--------------------------------------------------


In [6]:
# --- FINAL REPORT ---
print("\n" + "="*40)
print("📊 SUBMISSION GENERATION REPORT")
print("="*40)
print(f"{'Partition':<15} | {'Papers':<10} | {'Queries':<10} | {'MRR':<10}")
print("-" * 50)

for part in PARTITIONS:
    stats = summary_stats[part]
    print(f"{part.upper():<15} | {stats['papers']:<10} | {stats['queries']:<10} | {stats['mrr']:.4f}")

print("="*40)
print(f"\n📁 All pred.json files are ready at: {os.path.abspath(SUBMISSION_DIR)}")


📊 SUBMISSION GENERATION REPORT
Partition       | Papers     | Queries    | MRR       
--------------------------------------------------
TRAIN           | 292        | 6722       | 0.7942
VALIDATION      | 39         | 1127       | 0.8005
TEST            | 39         | 812        | 0.8349

📁 All pred.json files are ready at: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_final\23127011
